<a href="https://colab.research.google.com/github/gbulg/Data-analysis-class/blob/main/HW1/hw1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Homework 1

## 1. Read the novel text from txt file

Open the txt file, read from it and then close it.

In [130]:
f = open("./goncharov_obryv.txt", "r")
novel = f.read()
f.close()

Print first 500 characters to make sure everything is fine:

In [131]:
print(novel[:500])

Иван Александрович Гончаров

Обрыв

Роман в пяти частях.



ЧАСТЬ ПЕРВАЯ





I





Два господина сидели в небрежно убранной квартире в Петербурге, на одной из больших улиц. Одному было около тридцати пяти, а другому около сорока пяти лет.

Первый был Борис Павлович Райский, второй -- Иван Иванович Аянов.

У Бориса Павловича была живая, чрезвычайно подвижная физиономия. С первого взгляда он казался моложе своих лет: большой белый лоб блистал свежестью, глаза менялись, то загорались мыслию, чувс


## 2. Lemmatize text using Mystem and save it

First, import Mystem and create an instance of Mystem class:

In [132]:
from pymystem3 import Mystem

m = Mystem()

Lemmatize the text, and print a little slice of the obtained lemmas list:

In [133]:
lemmas = m.lemmatize(novel)

In [135]:
lemmas[50:60]

['квартира', ' ', 'в', ' ', 'петербург', ', ', 'на', ' ', 'один', ' ']

Now we need to save this lemmatization into the new txt file. I'm not sure how exactly it should be formatted, so I will just join this normal forms of words into new text and save it like that.

Open the file for writing, write into it and close it:

In [136]:
f = open("goncharov_obryv_lemmas.txt", "w")
f.write(''.join(lemmas))
f.close()

The saved text looks like this:

In [142]:
print(''.join(lemmas)[84:284])

два господин сидеть в небрежно убирать квартира в петербург, на один из большой улица. один быть около тридцать пять, а другой около сорок пять год.

первый быть борис павлович райский, второй -- иван


### 3.1. Tokenizing with NLTK

In [46]:
import nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


True

In [47]:
tokens = nltk.word_tokenize(novel)

In [103]:
tokens[100:110]

['.', 'Иногда', 'же', 'смотрели', 'они', 'зрело', ',', 'устало', ',', 'скучно']

### 3.2 Analysing tokens with pymorphy


In [53]:
from pymorphy2 import MorphAnalyzer
morph = MorphAnalyzer()

In [87]:
def token_to_dict(token):
  analysis = morph.parse(token)[0]
  return {"lemma": analysis.normal_form, "word": analysis.word, "pos": analysis.tag.POS}

tokens_analysis = list(map(token_to_dict, tokens))

In [107]:
len(tokens_analysis)

306236

In [108]:
tokens_analysis = list(filter(lambda tkn: tkn["pos"] != None, tokens_analysis))

In [109]:
len(tokens_analysis)

226709

In [110]:
tokens_analysis[100:110]

[{'lemma': 'этот', 'word': 'эти', 'pos': 'ADJF'},
 {'lemma': 'неизгладимый', 'word': 'неизгладимые', 'pos': 'ADJF'},
 {'lemma': 'знак', 'word': 'знаки', 'pos': 'NOUN'},
 {'lemma': 'время', 'word': 'времени', 'pos': 'NOUN'},
 {'lemma': 'и', 'word': 'и', 'pos': 'CONJ'},
 {'lemma': 'опыт', 'word': 'опыта', 'pos': 'NOUN'},
 {'lemma': 'гладкий', 'word': 'гладкие', 'pos': 'ADJF'},
 {'lemma': 'чёрный', 'word': 'чёрные', 'pos': 'ADJF'},
 {'lemma': 'волос', 'word': 'волосы', 'pos': 'NOUN'},
 {'lemma': 'падать', 'word': 'падали', 'pos': 'VERB'}]

### Task 4

In [121]:
parts_of_speech = set(map(lambda tkn: tkn["pos"], tokens_analysis))

for pos in parts_of_speech:
  share = len(list(filter(lambda tkn: tkn["pos"] == pos, tokens_analysis)))/len(tokens_analysis)
  print(pos + ": " + str(share))

INTJ: 0.001323282269340873
ADVB: 0.06901799222792214
PRCL: 0.06582447101791283
ADJS: 0.00862338945520469
PRTF: 0.006550247233237322
CONJ: 0.11151740777825318
VERB: 0.15375657781561386
PRTS: 0.0017511435364277554
NPRO: 0.11709283707307606
PREP: 0.1009311496235262
NOUN: 0.22016770397293448
PRED: 0.0064796721788724755
INFN: 0.02794772152847924
ADJF: 0.08848347440992638
COMP: 0.003956613985329211
GRND: 0.012090389000877777
NUMR: 0.00448592689306556


# TASK


1. Choose a novel or any other long text, e.g., from http://lib.ru/ and save it in the .txt format. Check that the encoding is UTF-8

2. (2 points) Lemmatize the text using mystem (if you absolutely do not want to use mystem, you can use any other suitable tool we have discussed) and save the result into a .txt file

3. (3 points)
tokenize the text using nltk

analyze the words using pymorphy

save the results of the analysis in jsonlines (.jsonl): every line contains morphological analysis in a form of a dictionary {"lemma": "конь", "word": "коня", "pos": "NOUN"}
4. (2 points) Answer the following questions:

What percentage constitutes each pos? (E.g., for the verb, the number of verbs divided by the total number of words)

Print out top-20 verbs and adverbs

you can keep the stop words or you can get rid of them
5. (1 point) Find top-25 bigrams and trigrams for your text (use nltk.bigrams), use only lemmas, get rid of the punctuation. Comment shortly on the results.

6. (2 points) Take 3-8 sentences from the original text and substitute some morphological information, for example, change the tense of the verbs, the number of the nouns, e.g, the original Слон подарил мартышке цветы should become Слоны подарят мартышкам цветок.